In [1]:
pip install langchain faiss-cpu sentence-transformers pypdf

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain-together

In [3]:
pip install --upgrade langchain-core langchain langchain-community langchain-together

Note: you may need to restart the kernel to use updated packages.


In [4]:
# main code

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load PDF
loader = PyPDFLoader("constitution of nepal.pdf")
pages = loader.load()

# Split into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(pages)

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\Dell\AppData\Local\Temp\ipykernel_15992\3745702769.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [6]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, embedding_model)
vectorstore.save_local("law_vectorstore")


In [7]:
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

llm = Ollama(model="mistral")
retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

C:\Users\Dell\AppData\Local\Temp\ipykernel_15992\2918871025.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")


In [8]:
pip install duckduckgo-search

In [9]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def create_vector_store(pdf_path):
    # Load PDF
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # Chunk text
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = splitter.split_documents(pages)

    # Embed
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Create vector DB
    vectorstore = FAISS.from_documents(docs, embedding_model)
    vectorstore.save_local("law_vectorstore")
    print("Vector store created and saved.")

create_vector_store("constitution of nepal.pdf")


Vector store created and saved.


In [10]:
def get_vector_context(query):
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.load_local("law_vectorstore", embedding_model)
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    docs = retriever.get_relevant_documents(query)
    
    return [doc.page_content for doc in docs]


In [11]:
from duckduckgo_search import DDGS

def perplexity_search(query):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
        return [res['body'] for res in results]


In [12]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key="442f97a9bf348a0486f931a5c86538c273ba620dc9c75e8c138545acec7c6859",  # Replace with your key
    model="mistralai/Mistral-7B-Instruct-v0.2"
)


C:\Users\Dell\AppData\Local\Temp\ipykernel_15992\4101164401.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [13]:
# test 1

In [22]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_together import Together
from langchain.prompts import PromptTemplate

# Step 1: Load your PDF
loader = PyPDFLoader("constitution of nepal.pdf")
documents = loader.load()

# Step 2: Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)  # Increased overlap
texts = text_splitter.split_documents(documents)

# Step 3: Load HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": False}
)

# Step 4: Store in FAISS
vectorstore = FAISS.from_documents(texts, embeddings)

# Step 5: Create retriever with more documents
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # Retrieve more documents

# Step 6: Custom prompt template for detailed answers
prompt_template = """Use the following pieces of context to provide a thorough, detailed answer to the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {question}

Provide a comprehensive answer with relevant details, examples, and explanations:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Step 7: Configure Together AI LLM for longer responses
llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.1",
    temperature=0.7,  # Slightly higher for more creative answers
    max_tokens=1024,  # Double the token limit
    top_p=0.9,       # For more diverse responses
    together_api_key="442f97a9bf348a0486f931a5c86538c273ba620dc9c75e8c138545acec7c6859"
)

# Step 8: Create RAG chain with custom prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},  # Use our detailed prompt
    verbose=True  # To see what's happening under the hood
)

# Step 9: Ask a question with more specific phrasing
query = """Explain in comprehensive detail what rights I have as a passenger if my bus is cancelled, 
including any relevant constitutional provisions, consumer protections, and practical steps I can take. 
Provide examples where applicable."""
response = qa_chain({"query": query})

print("Detailed Answer:", response["result"])
print("\nSupporting Documents:")
for i, doc in enumerate(response["source_documents"], 1):
    print(f"\nDocument {i}:")
    print("Source:", doc.metadata["source"])
    print("Content:", doc.page_content[:500] + "...")  # Show first 500 chars



> Entering new RetrievalQA chain...

> Finished chain.
Detailed Answer: 

If your bus is cancelled, you have several rights as a passenger. Firstly, you have the right to obtain quality goods and services under the Constitution of Nepal. This means that you have the right to receive a bus service that is reliable and efficient. If the bus is cancelled, this may be considered a breach of this right.

Secondly, you have the right to obtain compensation in case of injury from any substandard goods or services. If the bus cancellation causes you to miss an important appointment or event, you may be entitled to compensation for the inconvenience and loss caused.

Thirdly, you have the right against exploitation under the Constitution of Nepal. This means that you have the right to be treated fairly and with respect by the bus company and its employees. If the bus cancellation is due to the negligence or mismanagement of the bus company, this may be considered exploitation.

Fourthly, you 